## Génération du Markov Decision Process

### Définition du problème :

Un agent se trouve sur une case aléatoire d'une grille 4 $\times$ 4, sur la majorité des case de cette grille se trouvent des malus de score qui sont appliqués lorsqu'on marche dessus. Sur une faible fraction des cases se trouve un trésor, qui offre une récompense. Le but de l'agent est de maximiser sa récompense, en outre, il doit trouver le plus court chemin vers le trésor. Voici la grille des états :

$$grid = \begin{bmatrix}
1 & 2 & 3 & 4 \\
5 & 0 & 7 & 8 \\
9 & 10 & 11 & 12 \\
0 & 14 & 15 & 16 
\end{bmatrix}$$

On remarque que certains états sont impossibles, il y a des obstacles dessus, ce qui est traduit par une étoile sur la case correspondante.

D'après la définition du problème, la fonction de récompense ne dépend que de la case sur laquelle l'agent se déplace à un certain instant. On a donc $R:S\to \mathbb{R}$. Voici la matrice de récompense :
$$R=\begin{bmatrix}
-0.1 & -0.1 & -0.1 & -0.1 \\
- 1 & 0 & -0.1 & -0.1 \\
-0.1 & -0.1 & -0.1 & -0.1 \\
0 & 1 & -0.1 & -0.1
\end{bmatrix}$$

Les actions que peut prendre l'agent sont des déplacements dans les 4 directions, cependant, ces actions ne sont pas déterministes, l'agent ne peut que "essayer" d'aller dans une direction. Toutefois, s'il essaie d'aller dans une direction, il n'est pas possible qu'il aille à l'opposé. Par exemple, si l'agent prend l'action "essayer d'aller en haut", il n'est pas possible qu'il aille en bas, par contre il peut aller à gauche ou à droite avec une probabilité faible.





### Import des librairies



In [0]:
import numpy as np

### Définition des actions

In [0]:
actions = {'up' : [0.8,0.1,0.,0.1], 'right':[0.1,0.8,0.1,0], 'down':[0.,0.1,0.8,0.1], 'left':[0.1,0.,0.1,0.8]}

### Définition de la matrice représentative de la grille

In [0]:
grid = np.array([[1,2,3,4],[5,0,7,8],[9,10,11,12],[0,14,15,16]])
n_col = grid.shape[1]

### Définition de la matrice d'adjacence

In [0]:
n_states = np.prod(grid.shape)
adj = np.zeros((n_states,n_states))

for i in range (0, n_states) :
  for j in range (0, n_states) :
    # Si on est sur un état adjacent, et que cet état n'est pas obstrué, on met un 1 qui signifie qu'on peut aller sur cet état à partir
    # de l'état où l'on est
    if (j == i - n_col or j == i+1 or j == i + n_col or j == i-1) and grid[j//n_col,j%n_col]!=0 :
      adj[i,j] = 1

print(adj)

[[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]]


Une fois que l'on a la matrice d'adjacence, les matrices de transitions ne sont que des déclinaisons de cette matrice.
### Définition des matrices de transition

On considère que quelque soit l'état, on peut essayer d'aller dans toutes les directions. Cependant, les matrices de transitions doivent être stochastiques donc on modifie leur contenu de sorte que la somme de chaque ligne vale 1.

In [0]:
T=[]
adj_coor = [-n_col, 1,n_col, -1]
for action in actions :
  Ta = np.zeros((n_states,n_states))
  p = actions[action]
  for i in range (0, n_states) :
    for k in range(len(p)):
      j = i+adj_coor[k]
      if 0<=j<=15:
        Ta[i, j] = adj[i, j]*p[k]
    Lsum = np.sum(Ta[i,:])
    if Lsum !=1:
      Ta[i,:]*=1/Lsum
  T.append(Ta)
  
T=np.array(T)

[[0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.5        0.         0.5        0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.5        0.         0.5        0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.5        0.         0.5        0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.88888889 0.         0.         0.11111111 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.8        0.         0.         0.1        0.
  0.1        0.         0.         0.         0.         0.
  0.         0.         0.         0.        

### Définition des matrices de récompense

In [0]:
R = np.array([-0.1,-0.1,-0.1,-0.1,-1,0,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0,1,-0.1,-0.1]).transpose()
print(R)

[-0.1 -0.1 -0.1 -0.1 -1.   0.  -0.1 -0.1 -0.1 -0.1 -0.1 -0.1  0.   1.
 -0.1 -0.1]
